Goal: Retrieve the species checklist for a region using the eBird API.


In [1]:
from __future__ import annotations

In [13]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="eBird Checklists Notebook", timeout=10)
location = geolocator.geocode("Providence, Rhode Island")
location

Location(Providence, Providence County, Rhode Island, United States, (41.8239891, -71.4128343, 0.0))

In [14]:
print(location.address)
print(location.latitude)
print(location.longitude)

Providence, Providence County, Rhode Island, United States
41.8239891
-71.4128343


In [15]:
import os

from dotenv import load_dotenv

load_dotenv()  # Read variables from a .env file and set them in os.environ
ebird_api_key = os.getenv("EBIRD_API_KEY")
assert ebird_api_key

In [16]:
from ebird.api.requests import get_nearby_hotspots

nearby_hotspots_raw = get_nearby_hotspots(
    token=ebird_api_key, lat=location.latitude, lng=location.longitude
)

In [17]:
nearby_hotspots_raw

[{'locId': 'L9189645',
  'locName': 'Allen Harbor Marina',
  'countryCode': 'US',
  'subnational1Code': 'US-RI',
  'subnational2Code': 'US-RI-009',
  'lat': 41.6197094,
  'lng': -71.4112937,
  'latestObsDt': '2025-11-04 15:39',
  'numSpeciesAllTime': 121},
 {'locId': 'L5070824',
  'locName': "Allin's Cove (Barrington Land Trust)",
  'countryCode': 'US',
  'subnational1Code': 'US-RI',
  'subnational2Code': 'US-RI-001',
  'lat': 41.7442266,
  'lng': -71.3481226,
  'latestObsDt': '2025-11-02 14:30',
  'numSpeciesAllTime': 123},
 {'locId': 'L33244456',
  'locName': 'Anthony Lawrence Wildlife Preserve',
  'countryCode': 'US',
  'subnational1Code': 'US-MA',
  'subnational2Code': 'US-MA-005',
  'lat': 41.92324,
  'lng': -71.349151,
  'latestObsDt': '2025-05-03 10:07',
  'numSpeciesAllTime': 55},
 {'locId': 'L634993',
  'locName': 'Apponaug Cove',
  'countryCode': 'US',
  'subnational1Code': 'US-RI',
  'subnational2Code': 'US-RI-003',
  'lat': 41.6953469,
  'lng': -71.4527607,
  'latestObsDt':

In [18]:
from dataclasses import dataclass
from typing import Any

Coordinate = tuple[float, float]


@dataclass(frozen=True)
class EBirdLocation:
    """A location in the eBird system."""

    id: str
    name: str
    latitude: float
    longitude: float
    country_code: str
    subnational1_code: str | None
    subnational1_name: str | None
    subnational2_code: str | None
    subnational2_name: str | None

    @classmethod
    def from_dict(cls, data: dict[str, Any]) -> EBirdLocation:
        """Construct an eBird location from a dictionary of data."""
        return EBirdLocation(
            id=data["locId"],
            name=data["locName"],
            latitude=data["lat"],
            longitude=data["lng"],
            country_code=data["countryCode"],
            subnational1_code=data.get("subnational1Code"),
            subnational1_name=data.get("subnational1Name"),
            subnational2_code=data.get("subnational2Code"),
            subnational2_name=data.get("subnational2Name"),
        )

    @property
    def coordinate(self) -> Coordinate:
        """Retrieve the (latitude, longitude) GPS coordinate of the location."""
        return (self.latitude, self.longitude)


@dataclass(frozen=True)
class EBirdHotspot:
    """A location in the eBird system with a history of bird observations."""

    location: EBirdLocation
    all_time_species: int

    @classmethod
    def from_dict(cls, data: dict[str, Any]) -> EBirdHotspot:
        """Construct an eBird hotspot from a dictionary of data."""
        location = EBirdLocation.from_dict(data)
        return EBirdHotspot(location=location, all_time_species=data["numSpeciesAllTime"])


nearby_hotspots = [EBirdHotspot.from_dict(data) for data in nearby_hotspots_raw]
nearby_hotspots

[EBirdHotspot(location=EBirdLocation(id='L9189645', name='Allen Harbor Marina', latitude=41.6197094, longitude=-71.4112937, country_code='US', subnational1_code='US-RI', subnational1_name=None, subnational2_code='US-RI-009', subnational2_name=None), all_time_species=121),
 EBirdHotspot(location=EBirdLocation(id='L5070824', name="Allin's Cove (Barrington Land Trust)", latitude=41.7442266, longitude=-71.3481226, country_code='US', subnational1_code='US-RI', subnational1_name=None, subnational2_code='US-RI-001', subnational2_name=None), all_time_species=123),
 EBirdHotspot(location=EBirdLocation(id='L33244456', name='Anthony Lawrence Wildlife Preserve', latitude=41.92324, longitude=-71.349151, country_code='US', subnational1_code='US-MA', subnational1_name=None, subnational2_code='US-MA-005', subnational2_name=None), all_time_species=55),
 EBirdHotspot(location=EBirdLocation(id='L634993', name='Apponaug Cove', latitude=41.6953469, longitude=-71.4527607, country_code='US', subnational1_cod

In [19]:
from geopy.distance import geodesic

# Find the hotspot nearest to the original location
location_coord = (location.latitude, location.longitude)


def geodesic_distance_mi(c1: Coordinate, c2: Coordinate) -> float:
    """Find the geodesic distance (in miles) between two GPS coordinates."""
    return geodesic(c1, c2).miles


nearest_hotspot = min(
    nearby_hotspots,
    key=lambda hs: geodesic_distance_mi(c1=location_coord, c2=hs.location.coordinate),
)
nearest_hotspot

EBirdHotspot(location=EBirdLocation(id='L24184797', name='Kennedy Plaza, Providence', latitude=41.825473, longitude=-71.412127, country_code='US', subnational1_code='US-RI', subnational1_name=None, subnational2_code='US-RI-007', subnational2_name=None), all_time_species=38)

In [20]:
from ebird.api.requests import get_species_list

get_species_list(token=ebird_api_key, area=nearest_hotspot.location.id)

['cangoo',
 'mutswa',
 'mallar3',
 'ambduc',
 'wiltur',
 'rocpig',
 'moudov',
 'chiswi',
 'ribgul',
 'amhgul1',
 'doccor',
 'coohaw',
 'rebwoo',
 'dowwoo',
 'perfal',
 'easpho',
 'blujay',
 'amecro',
 'fiscro',
 'bkcchi',
 'tuftit',
 'gockin',
 'whbnut',
 'carwre',
 'eursta',
 'grycat',
 'normoc',
 'amerob',
 'cedwax',
 'houspa',
 'houfin',
 'amegfi',
 'chispa',
 'fiespa',
 'sonspa',
 'bnhcow',
 'comgra',
 'norcar']

In [32]:
@dataclass(frozen=True)
class SurroundingSpecies:
    """A collection of species information in the regions containing a location."""

    location: EBirdLocation
    location_species: list[str]
    subnational2_species: list[str] | None
    subnational1_species: list[str] | None
    country_species: list[str]

    def __str__(self) -> str:
        """Return a human-readable string representation of the species collection."""
        species_str = f"Location ({self.location.name}) Species: {len(self.location_species)}\n"

        if self.subnational2_species is not None:
            subnat2_name = (
                self.location.subnational2_name
                if self.location.subnational2_name is not None
                else self.location.subnational2_code
            )
            n_subnat2_species = len(self.subnational2_species)
            species_str += f"Subnational Region 2 ({subnat2_name}) Species: {n_subnat2_species}\n"

        if self.subnational1_species is not None:
            subnat1_name = (
                self.location.subnational1_name
                if self.location.subnational1_name is not None
                else self.location.subnational1_code
            )
            n_subnat1_species = len(self.subnational1_species)
            species_str += f"Subnational Region 1 ({subnat1_name}) Species: {n_subnat1_species}\n"

        country_code = self.location.country_code
        species_str += f"Country ({country_code}) Species: {len(self.country_species)}"

        return species_str

    @classmethod
    def from_location(cls, location: EBirdLocation) -> SurroundingSpecies:
        """Identify the species in the various regions surrounding the given location."""
        location_species = get_species_list(ebird_api_key, location.id)

        if location.subnational1_code is not None:
            try:
                subnat1_species = get_species_list(ebird_api_key, location.subnational1_code)
            except ValueError:
                subnat1_species = None
        else:
            subnat1_species = None

        if location.subnational2_code is not None:
            try:
                subnat2_species = get_species_list(ebird_api_key, location.subnational2_code)
            except ValueError:
                subnat2_species = None
        else:
            subnat2_species = None

        country_species = get_species_list(ebird_api_key, location.country_code)

        return SurroundingSpecies(
            location, location_species, subnat2_species, subnat1_species, country_species
        )

In [33]:
species = SurroundingSpecies.from_location(nearest_hotspot.location)
species

SurroundingSpecies(location=EBirdLocation(id='L24184797', name='Kennedy Plaza, Providence', latitude=41.825473, longitude=-71.412127, country_code='US', subnational1_code='US-RI', subnational1_name=None, subnational2_code='US-RI-007', subnational2_name=None), location_species=['cangoo', 'mutswa', 'mallar3', 'ambduc', 'wiltur', 'rocpig', 'moudov', 'chiswi', 'ribgul', 'amhgul1', 'doccor', 'coohaw', 'rebwoo', 'dowwoo', 'perfal', 'easpho', 'blujay', 'amecro', 'fiscro', 'bkcchi', 'tuftit', 'gockin', 'whbnut', 'carwre', 'eursta', 'grycat', 'normoc', 'amerob', 'cedwax', 'houspa', 'houfin', 'amegfi', 'chispa', 'fiespa', 'sonspa', 'bnhcow', 'comgra', 'norcar'], subnational2_species=['snogoo', 'rosgoo', 'gragoo', 'x00776', 'gwfgoo', 'pifgoo', 'brant', 'bargoo', 'cacgoo1', 'cangoo', 'x00758', 'x00759', 'mutswa', 'truswa', 'tunswa', 'musduc', 'wooduc', 'buwtea', 'cintea', 'norsho', 'gadwal', 'eurwig', 'amewig', 'x00724', 'mallar3', 'ambduc', 'x00004', 'norpin', 'x00628', 'gnwtea', 'canvas', 'redhe

In [34]:
print(species)

Location (Kennedy Plaza, Providence) Species: 38
Subnational Region 2 (US-RI-007) Species: 328
Subnational Region 1 (US-RI) Species: 472
Country (US) Species: 1775
